# Lab 8 - Integer Programming - BnB for MIP

Information on group members:

1) Student ID, Name and last name <br>
2) 148257, Daniel Jankowski

In [1]:
from pulp import *  
import numpy as np
import pandas as pd

1) Given is the below MIP problem. Note that the first 5 variables are of an integer type with specified upper bounds

In [2]:
def getProblem(relaxed = False):
    
    A = [
        [0,3,2,0,0,0,-3,-1,0,0],
        [1,1,0,2,0,0,0,-1,2,1],
        [0,0,2,-2,3,0,-2,2,1,0],
        [0,0,2,0,0,-1,0,0,0,1],
        [0,2,0,0,0,-2,0,0,0,1],
        [1,4,0,0,0,0,-3,6,2,0],
        [2,2,0,0,2,2,0,0,2,2],
        [0,0,3,0,-1,1,0,-1,0,1],
        [0,0,0,0,5,0,1,1,0,3],
        [2,-7,0,0,0,1,0,8,2,0]]
    b = [10,15,20,20,30,50,40,20,25,25]
    c = [5, 7, 5, 5, 5, 5, 7, 4, 9, 10]
    uB = [5, 8, 4, 5, 4, 5, 5, 3, 3, 3]
    
    problem = LpProblem(name="bnb-problem", sense=LpMaximize)
    
    ### 5 integers and 3 continuous (if relaxed, 8 cont.)
    cat = ['Integer' for i in range(5)] + ['Continuous' for i in range(5)]
    if relaxed: cat = ['Continuous' for i in range(5)] + ['Continuous' for i in range(5)]
        
    x = [LpVariable(name="x"+ str(i+1), lowBound=0, upBound=uB[i], cat = cat[i]) for i in range(10)]
    
    for r in range(10):
        expr = lpSum([x[j] * A[r][j] for j in range(10)])
        problem += LpConstraint(e=expr, sense = -1, name = "baseC"+str(r+1), rhs = b[r])
        
    obj_func = lpSum([x[j] * c[j] for j in range(10)])
    problem += obj_func
    
    return x, problem

x, P = getProblem()
print(P)

bnb-problem:
MAXIMIZE
5*x1 + 10*x10 + 7*x2 + 5*x3 + 5*x4 + 5*x5 + 5*x6 + 7*x7 + 4*x8 + 9*x9 + 0
SUBJECT TO
baseC1: 3 x2 + 2 x3 - 3 x7 - x8 <= 10

baseC2: x1 + x10 + x2 + 2 x4 - x8 + 2 x9 <= 15

baseC3: 2 x3 - 2 x4 + 3 x5 - 2 x7 + 2 x8 + x9 <= 20

baseC4: x10 + 2 x3 - x6 <= 20

baseC5: x10 + 2 x2 - 2 x6 <= 30

baseC6: x1 + 4 x2 - 3 x7 + 6 x8 + 2 x9 <= 50

baseC7: 2 x1 + 2 x10 + 2 x2 + 2 x5 + 2 x6 + 2 x9 <= 40

baseC8: x10 + 3 x3 - x5 + x6 - x8 <= 20

baseC9: 3 x10 + 5 x5 + x7 + x8 <= 25

baseC10: 2 x1 - 7 x2 + x6 + 8 x8 + 2 x9 <= 25

VARIABLES
0 <= x1 <= 5 Integer
x10 <= 3 Continuous
0 <= x2 <= 8 Integer
0 <= x3 <= 4 Integer
0 <= x4 <= 5 Integer
0 <= x5 <= 4 Integer
x6 <= 5 Continuous
x7 <= 5 Continuous
x8 <= 3 Continuous
x9 <= 3 Continuous



2) The below function returns None if the problem has no feasible solutions. Otherwise, it returns a tuple: objective function values and a vector of decision variables. 

In [3]:
def getSolution(x, problem):
    status = problem.solve()
    if problem.status != 1: return None
    return problem.objective.value(), [_.value() for _ in x]

3) PuLP can solve MIP problems. Hence, the "relaxed" flag can be set to False. Solve the problem and analyze the obtained outcome.  

In [4]:
x, problem = getProblem(relaxed = False)
print(getSolution(x, problem))

(207.0, [3.0, 6.0, 4.0, 1.0, 1.0, 5.0, 5.0, 3.0, 2.0, 3.0])


4) Now, compare this solution with the one obtained for the relaxed LP problem: 

In [5]:
x, problem = getProblem(relaxed = True)
print(getSolution(x, problem))

(211.33333355000002, [0.73333333, 6.6666667, 4.0, 0.8, 1.6, 5.0, 5.0, 3.0, 3.0, 3.0])


5) Your task is to implement the Branch and Bound Algorithm for solving MIP problems. You can use the PuLP library for solving the relaxed LP subproblems. <br>
<ul> 
    <li> Firstly, as a node selection policy, implement the default DFS-like strategy as shown in the lecture (generate both children in one iteration; prioritize the left children, i.e., associated with the "<=" constraint). As for the variable selection policy, take the one with the lowest index  (default, arbitrary selection). 
<li> Identify how many LP relaxed problems have to be solved to find the optimum. Note that such a number was reported to be 35 for the default policies. However, it may vary slightly for different solvers due to possible multiple sub-optima.
<li> Propose at least 3 new node and variable selection policies with the aim of minimizing the number of solver runs required to reach the optimum.  Try getting below 20. 
    </ul>

In [71]:
from math import floor, ceil

var_mip, _ = getProblem(relaxed=False) # get MIP problem to save information about integer/continuous variables
integer_vars_inx = [i for i in range(len(var_mip)) if var_mip[i].cat=="Integer"] # indiced of integer variables

x, P = getProblem(relaxed=True) # get initial relaxed problem to be solved as a first step

# Create a queue to store the nodes of the branch and bound tree
nodes = []

#best found objective value and variables values
best_solution_value = -float('inf')
best_solution_vars = []

# Add the root node to the queue
nodes.append(P)

# Keep track of the number of LP relaxed problems solved
subproblems_solved = 0

while nodes:
    # Pop the next node from the queue
    node = nodes.pop()
    # Check if the LP relaxation has a feasible solution
    if getSolution(x, node):
        
        problem_objective, vars_values = getSolution(x, node)
        subproblems_solved += 1
        
        # Check if the LP relaxation has an integer feasible solution
        if all(vars_values[inx].is_integer() for inx in integer_vars_inx):
            if problem_objective > best_solution_value:
                best_solution_value = problem_objective
                best_solution_vars = vars_values
        else:
            
            # feasible solution but it is not integer
            var_to_branch_on = None
            for inx in integer_vars_inx: 
                if not vars_values[inx].is_integer(): # take first possible variable which should has integer value but doesn't have
                    var_to_branch_on = inx
                    break
            if var_to_branch_on is not None:
                # Generate the right child (associated with the ">=" constraint)
                right_child = node.copy()
                right_child += x[inx] >= ceil(x[inx].value())
                nodes.append(right_child)
                
                # Generate the left child (associated with the "<=" constraint)
                left_child = node.copy()
                left_child += x[inx] <= floor(x[inx].value())
                nodes.append(left_child)
                # add left child at the end, because we want to prioritize left one (this one will be poped from the list earlier)

print("Number of LP relaxed problems solved:", subproblems_solved)

print("Optimal solution value:", best_solution_value)
print("Variables values: ", best_solution_vars)


Number of LP relaxed problems solved: 96
Optimal solution value: 207.0
Variables values:  [3.0, 6.0, 4.0, 1.0, 1.0, 5.0, 5.0, 3.0, 2.0, 3.0]


In [ ]:
#TODO: Trzy inne opcje splitowania
#TODO: debugging bo duzo iteracji xd
